In [57]:
import pytorch_lightning as pl
pytorch_lightning.__version__

'1.2.10'

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.8.1'

In [81]:
#dataset
from sklearn.datasets import load_breast_cancer

In [82]:
breast_cancer = load_breast_cancer()
x = breast_cancer['data']
t = breast_cancer['target']

In [83]:
#cahnge Trnsor
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [84]:
#dataset 
dataset = torch.utils.data.TensorDataset(x,t)
dataset

In [85]:
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val


pl.seed_everything(0)


train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

Global seed set to 0


In [86]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True,drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [87]:
#define
#nn.Module - pl.LightningModule
#__init__ forward
#traing_step configue_optimizer

In [94]:
class Net(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(30,10)
        self.bn1 = nn.BatchNorm1d(10)
        self.fc2 = nn.Linear(10,2)
        
        
    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.bn1(h)
        h = self.fc2(h)
        return h 
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr = 0.01)
        return optimizer

In [97]:
#train network

pl.seed_everything(0)
net = Net()
trainer = pl.Trainer(max_epochs=30,)
trainer.fit(net, train_loader)

Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)


1

In [ ]:
### status

In [99]:
x, t = next(iter(test_loader))
x.shape, t.shape

(torch.Size([32, 30]), torch.Size([32]))

In [100]:
y = net(x)
y

tensor([[ 0.2605, -0.5296],
        [-2.0603,  0.9093],
        [-0.6161,  0.0065],
        [-1.7466,  0.6988],
        [-1.4331,  0.5453],
        [-1.4653,  0.5530],
        [-1.3732,  0.5662],
        [ 2.9437, -2.3173],
        [-1.8469,  0.6623],
        [ 1.6638, -1.4024],
        [ 0.6856, -1.0034],
        [-1.1796,  0.3917],
        [-1.3056,  0.4868],
        [-1.0606,  0.3132],
        [-1.2155,  0.3994],
        [ 1.4462, -1.3221],
        [-1.1211,  0.3836],
        [ 1.6782, -1.5773],
        [-1.2297,  0.3528],
        [ 1.0574, -0.9348],
        [-1.0798,  0.3374],
        [ 5.2169, -3.7300],
        [-2.0156,  0.8760],
        [-1.4219,  0.4708],
        [-2.0414,  0.8575],
        [-2.1125,  0.8560],
        [-0.4463, -0.0123],
        [-0.7372,  0.1194],
        [ 2.3008, -1.8526],
        [-0.1567, -0.2762],
        [-1.3378,  0.4909],
        [-1.7875,  0.7240]], grad_fn=<AddmmBackward>)

In [101]:
y_label = torch.argmax(y, dim=1)
y_label

tensor([0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 0, 1, 1])

In [102]:
t

tensor([0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 0, 0, 1, 1])

In [105]:
accuracy = (y_label == t).sum() / len(t)
accuracy

tensor(0.9688)

In [109]:
from pytorch_lightning.metrics.functional import accuracy

In [ ]:
#log memory

In [120]:
############# tain $ test 

In [123]:
class Net(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(30,10)
        self.bn1 = nn.BatchNorm1d(10)
        self.fc2 = nn.Linear(10,2)
        
        
    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.bn1(h)
        h = self.fc2(h)
        return h 
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('train_loss', loss, on_step = False, on_epoch=True, prog_bar=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('val_loss', loss, on_step = False, on_epoch=True, prog_bar=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('test_loss', loss, on_step = False, on_epoch=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr = 0.01)
        return optimizer

In [124]:
pl.seed_everything(0)
net = Net()
trainer = pl.Trainer(max_epochs=30,)
trainer.fit(net, train_loader, val_loader)

Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)
/Users/kitamurahiroto/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/Users/kitamurahiroto/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

In [125]:
#test data trainer.callback_metrics

In [128]:
results = trainer.test(test_dataloaders=test_loader)
results


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.20053799450397491}
--------------------------------------------------------------------------------


[{'test_loss': 0.20053799450397491}]

In [129]:
trainer.callback_metrics

{'val_loss': tensor(0.2253),
 'train_loss': tensor(0.2463),
 'test_loss': tensor(0.2005)}

In [130]:
%tensorboard --logdir lightning_logs/